In [4]:
import datetime
import hashlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from decimal import Decimal

# Fonction de hashage

Un message est une chaine de caractère. L'ordinateur le stocke sous la forme d'un objet "bytes" caque byte comprend 8 bits qui sont des 0 et des 1.

In [5]:
# Un message sous la forme d'un objet bytes avec l'encodage ASCII 
b = b"Happy new year"
print(b)
# Les bytes sont des suite de byte qui sont des fait de 8 bits (des 0 et des 1)
# On peut convertir l'objet bytes en bits
binary_string = ''.join(['{:08b}'.format(byte) for byte in b])
print(binary_string)
bin(int(binary_string,2))

b'Happy new year'
0100100001100001011100000111000001111001001000000110111001100101011101110010000001111001011001010110000101110010


'0b100100001100001011100000111000001111001001000000110111001100101011101110010000001111001011001010110000101110010'

In [6]:
hashlib.sha256(b"Happy new year").hexdigest()
int(hashlib.sha256(b"Happy new year").hexdigest(), 16)
'%.2E' % Decimal(int(hashlib.sha256(b"Happy new year").hexdigest(), 16))

'1.06E+77'

# Preuve de travail

## La classe blocs

In [7]:
class Block:
    hash = None # Hash of the block info
    index = 0 # Index of the block inside the chain
    timestamp = datetime.datetime.now() # Time of creation of the block
    nonce = 0 # Solution to the cryptopuzzle
    transactions = [] # Transaction data
    mined = False # Boolean set to True whenever the problem has been solved
    previous_hash = 0x0 # Hash of the previous block
    

    def __init__(self, transactions):
        self.transactions = transactions
        self.timestamp = datetime.datetime.now()

    def hash(self):# Compute the hash of the blockdata
        h = hashlib.sha256()
        h.update(
        str(self.nonce).encode('utf-8') +
        str(self.transactions).encode('utf-8') +
        str(self.previous_hash).encode('utf-8') 
        )
        return h.hexdigest()
    
    # Add a new transaction to a block
    def new_transaction(self, sender, recipient, amount, fee):
        transaction = {
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'fee' : fee
        }
        self.transactions.append(transaction)
    
    # Print the block info
    def __str__(self):
        return "Block Height: " + str(self.index) + \
    "\nBlock Hash: " + str(self.hash()) + \
    "\nTime:" + str(self.timestamp) + \
    "\nBlock data: " + str(self.transactions) + \
    "\nMined: " + str(self.mined) + \
    "\nPrevious block hash: " + str(self.previous_hash) +"\n--------------"
    
    # Solve the cryptopuzzle of the block
    


In [8]:
B1 = Block([])
B1.new_transaction("Coinbase", "Satoshi", 100, 0)
B1.new_transaction("Satoshi", "Pierre-O", 5, 2)
print(B1)


Block Height: 0
Block Hash: 99f27c54043f4464a1f7a2ef0960a68c4ad07f3f0b863c57023b5a19efdf9a7c
Time:2023-01-07 16:48:14.978290
Block data: [{'sender': 'Coinbase', 'recipient': 'Satoshi', 'amount': 100, 'fee': 0}, {'sender': 'Satoshi', 'recipient': 'Pierre-O', 'amount': 5, 'fee': 2}]
Mined: False
Previous block hash: 0
--------------


In [9]:
class Blockchain:
    diff = 0
    maxNonce = 2**32
    target = 2 ** (256-diff)
    reward = 50
    miner = "Miner"

    def __init__(self, genesis_block):
        self.chain = [genesis_block]
        self.pending_transactions = []
        
    def new_transaction(self, sender, recipient, amount, fee):
        transaction = {
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'fee' : fee
        }
        self.pending_transactions.append(transaction) 
    
    def add_block(self):
#         blockchain.pending_transactions
        current_block = blockchain.chain[-1]
        new_block = Block(blockchain.pending_transactions)
        new_block.index = current_block.index + 1
        new_block.previous_hash = current_block.hash()
        blockchain.chain.append(new_block)
        blockchain.pending_transactions = []
    
    def adjust_difficulty(self, new_diff):
        self.diff = new_diff
        self.target = 2 ** (256-new_diff)
    
    def halve_reward(self):
        self.reward = self.reward / 2
    
    def mine(self):
        block = self.chain[-1]
        target = self.target

        if block.transactions:
            fee = pd.DataFrame.from_records(block.transactions).fee.sum()
        else:
            fee = 0

        block.new_transaction("Coinbase", self.miner, self.reward + fee, 0)
        while int(block.hash(), 16) > target:
            block.nonce = int(np.random.uniform(low = 0, high = 2**32 + 1))
        block.mined = True
        block.timestamp = datetime.datetime.now() 
        



In [10]:
genesis_block = Block([])
blockchain = Blockchain(genesis_block)
print(blockchain.chain[-1])
blockchain.adjust_difficulty(4)
blockchain.halve_reward()
blockchain.mine()
print(blockchain.chain[-1])

Block Height: 0
Block Hash: 32369d15916932bd1ade51c0cae9f32f5ded2cccd29d81d7eec27066c813e39d
Time:2023-01-07 16:49:03.575409
Block data: []
Mined: False
Previous block hash: 0
--------------
Block Height: 0
Block Hash: 06dfa5c67eba88e0132edf1c12282cd27c7f80e32f1e2b229395e81266046b0e
Time:2023-01-07 16:49:03.588240
Block data: [{'sender': 'Coinbase', 'recipient': 'Miner', 'amount': 25.0, 'fee': 0}]
Mined: True
Previous block hash: 0
--------------


In [11]:
blockchain.new_transaction("miner", "Pierre-O", 5, 0.1)
blockchain.new_transaction("miner", "Satoshi", 10, 0.2)
# print(blockchain.pending_transactions)
blockchain.add_block()
for block in blockchain.chain: 
    print(block)

Block Height: 0
Block Hash: 06dfa5c67eba88e0132edf1c12282cd27c7f80e32f1e2b229395e81266046b0e
Time:2023-01-07 16:49:03.588240
Block data: [{'sender': 'Coinbase', 'recipient': 'Miner', 'amount': 25.0, 'fee': 0}]
Mined: True
Previous block hash: 0
--------------
Block Height: 1
Block Hash: 1771264be11623c2cb7fa2fb67bdff56b22d72cb68c268b081c8ec27d0381f22
Time:2023-01-07 16:49:20.957757
Block data: [{'sender': 'miner', 'recipient': 'Pierre-O', 'amount': 5, 'fee': 0.1}, {'sender': 'miner', 'recipient': 'Satoshi', 'amount': 10, 'fee': 0.2}]
Mined: False
Previous block hash: 06dfa5c67eba88e0132edf1c12282cd27c7f80e32f1e2b229395e81266046b0e
--------------


In [12]:
blockchain.mine()
for block in blockchain.chain: 
    print(block)

Block Height: 0
Block Hash: 06dfa5c67eba88e0132edf1c12282cd27c7f80e32f1e2b229395e81266046b0e
Time:2023-01-07 16:49:03.588240
Block data: [{'sender': 'Coinbase', 'recipient': 'Miner', 'amount': 25.0, 'fee': 0}]
Mined: True
Previous block hash: 0
--------------
Block Height: 1
Block Hash: 0045d9ff07f1717a42c270a98874dc6015f9ebd9303286befc201f7e7585ee18
Time:2023-01-07 16:49:59.651288
Block data: [{'sender': 'miner', 'recipient': 'Pierre-O', 'amount': 5, 'fee': 0.1}, {'sender': 'miner', 'recipient': 'Satoshi', 'amount': 10, 'fee': 0.2}, {'sender': 'Coinbase', 'recipient': 'Miner', 'amount': 25.3, 'fee': 0}]
Mined: True
Previous block hash: 06dfa5c67eba88e0132edf1c12282cd27c7f80e32f1e2b229395e81266046b0e
--------------
